In [ ]:
# 看看决策树是什么

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits
from sklearn.metrics import r2_score,mean_absolute_error, mean_squared_error
from sklearn.tree import DecisionTreeRegressor,export_graphviz
import graphviz


DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [21]:
# Titanic 生存分析
import pandas as pd
from pandas import Series,DataFrame

# 数据加载
train_data = pd.read_csv('./Titanic_Data/train.csv')
test_data = pd.read_csv('./Titanic_Data/test.csv')

# 数据探索
## 了解数据表的基本情况：行数、列数、每列的数据类型、数据完整度；
print(train_data.info())
print('='*30)
## 了解数据表的统计情况：总数、平均值、标准差、最小值、最大值等；
print(train_data.describe())
print('='*30)
## 查看字符串类型（非数字）的整体情况；
print(train_data.describe(include=['O']))
print('='*30)
## 查看前几行数据（默认是前 5 行）；
print(train_data.head())
print('='*30)
## 查看后几行数据（默认是后5行）
print(train_data.tail())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.48659

In [28]:
# 数据清洗
# 使用平均年龄来填充年龄中的nan值
train_data['Age'].fillna(train_data['Age'].mean(), inplace=True)
test_data['Age'].fillna(test_data['Age'].mean(), inplace=True)
# 使用票价的均值填充票价中nan值
train_data['Fare'].fillna(train_data['Fare'].mean(), inplace=True)
test_data['Fare'].fillna(test_data['Fare'].mean(), inplace=True)

# Cabin为船舱，从Describe中可以看出缺失值较多，训练集和测试集中缺失率达到了77%和78%,无法补齐；
# Embarked为登录港口，有少量缺失值，可以补齐
# 先观察Embarked字段的取值
print(train_data['Embarked'].value_counts())
# 使用登录最多的港口来填充登录港口的nan值
train_data['Embarked'].fillna('S', inplace=True)
test_data['Embarked'].fillna('S',inplace=True)
print(train_data['Embarked'].value_counts())

S    646
C    168
Q     77
Name: Embarked, dtype: int64
S    646
C    168
Q     77
Name: Embarked, dtype: int64


In [30]:
# 特征选择
# 将Pclass、Sex、Age这些剩余字段作特征，放入特征向量features里
# 特征选择
features = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']
train_features = train_data[features]
train_labels = train_data['Survived']
test_features = test_data[features]

In [41]:
ylk_data = DataFrame(pd.read_excel('ylk.xlsx'))
print(ylk_data)
df = pd.melt(ylk_data,id_vars=['finance_no'],value_vars=['glod','platinum','diamond'])
df

# 导出到excel
df.to_excel('ylk_new.xls')
df

     finance_no                 glod                 platinum  \
0          0232  ylk_free_gold_close  ylk_free_platinum_close   
1          1512   ylk_free_glod_open   ylk_free_platinum_open   
2          0900  ylk_free_gold_close   ylk_free_platinum_open   
3          0887  ylk_free_gold_close  ylk_free_platinum_close   
4          2254  ylk_free_gold_close  ylk_free_platinum_close   
...         ...                  ...                      ...   
1628       1926   ylk_free_glod_open   ylk_free_platinum_open   
1629       1289   ylk_free_glod_open   ylk_free_platinum_open   
1630       1185  ylk_free_gold_close   ylk_free_platinum_open   
1631       1756   ylk_free_glod_open   ylk_free_platinum_open   
1632       0694  ylk_free_gold_close  ylk_free_platinum_close   

                    diamond  
0     ylk_free_diamond_open  
1     ylk_free_diamond_open  
2     ylk_free_diamond_open  
3     ylk_free_diamond_open  
4     ylk_free_diamond_open  
...                     ...  
1628  ylk

,finance_no,variable,value
0,0232,glod,ylk_free_gold_close
1,1512,glod,ylk_free_glod_open
2,0900,glod,ylk_free_gold_close
3,0887,glod,ylk_free_gold_close
4,2254,glod,ylk_free_gold_close
...,...,...,...
4894,1926,diamond,ylk_free_diamond_open
4895,1289,diamond,ylk_free_diamond_open
4896,1185,diamond,ylk_free_diamond_open
4897,1756,diamond,ylk_free_diamond_open


In [54]:
df = DataFrame(pd.read_excel('pos3_ylk.xlsx'))

df = df.pivot(index="finance_no",columns="variable",values="value")
df.to_excel('ylk_pos3_new.xls')
df

variable,diamond,gold,platinum
finance_no,,,
0002,ylk_free_diamond_open,ylk_free_gold_close,ylk_free_platinum_open
0003,ylk_free_diamond_open,ylk_free_gold_close,ylk_free_platinum_close
0004,ylk_free_diamond_open,ylk_free_gold_close,ylk_free_platinum_close
0005,ylk_free_diamond_open,ylk_free_gold_close,ylk_free_platinum_close
0006,ylk_free_diamond_open,ylk_free_gold_close,ylk_free_platinum_close
...,...,...,...
9991,ylk_free_diamond_open,ylk_free_gold_close,ylk_free_platinum_close
M0001,ylk_free_diamond_open,ylk_free_gold_close,ylk_free_platinum_close
M0004,ylk_free_diamond_open,ylk_free_gold_close,ylk_free_platinum_close


variable,diamond,gold,platinum
finance_no,,,
0002,ylk_free_diamond_open,ylk_free_gold_close,ylk_free_platinum_open
0003,ylk_free_diamond_open,ylk_free_gold_close,ylk_free_platinum_close
0004,ylk_free_diamond_open,ylk_free_gold_close,ylk_free_platinum_close
0005,ylk_free_diamond_open,ylk_free_gold_close,ylk_free_platinum_close
0006,ylk_free_diamond_open,ylk_free_gold_close,ylk_free_platinum_close
...,...,...,...
9991,ylk_free_diamond_open,ylk_free_gold_close,ylk_free_platinum_close
M0001,ylk_free_diamond_open,ylk_free_gold_close,ylk_free_platinum_close
M0004,ylk_free_diamond_open,ylk_free_gold_close,ylk_free_platinum_close
